I'm using the conda 3.12 environment when it's running locally.  I had to manually install the following.

```sh
conda env list
conda activate py3.12
conda install -c conda-forge python-dotenv
conda install pillow
pip install openai==1.1.1
conda install requests
```

In [ ]:
# import sys
# !{sys.executable} -m pip install openai==1.1.1
# !{sys.executable} -m pip install tiktoken
# !{sys.executable} -m pip install dotenv

In [ ]:
import sys
!{sys.executable} -m pip show openai

In [6]:
import os
import dotenv

# Get the path to the .env file
dotenv_path = dotenv.find_dotenv()

# Load the .env file
dotenv.load_dotenv(dotenv_path)

api_key = os.getenv('OPENAI_API_KEY')
# print(f'api_key: {api_key}')


In [7]:
import base64
import time
import openai
import os
import requests
import json

In [8]:
from openai import OpenAI
client = OpenAI(api_key=api_key)

GPT-4 Turbo is our latest generation model. It’s more capable, has an updated knowledge cutoff of April 2023 and introduces a 128k context window (the equivalent of 300 pages of text in a single prompt). The model is also 3X cheaper for input tokens and 2X cheaper for output tokens compared to the original GPT-4 model. The maximum number of output tokens for this model is 4096.

In [ ]:
completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "What are some of the key events that happened in March 2021?",
        }
    ],
    model="gpt-4-1106-preview",
)

In [ ]:
print(completion.choices[0].message.content)

In [ ]:
completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "What are some of the key events that happened Janurary 2023 for New Zealand Prime Minister Jacinda Ardern?",
        }
    ],
    model="gpt-3.5-turbo-1106",
)

In [ ]:
print(completion.choices[0].message.content)

JSON Mode

By setting response_format: { type: "json_object" } when calling gpt-4-visual-preview or gpt-3.5-turbo, these models will constrain their output to generate strings that parse into valid JSON. 

In [9]:
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model="gpt-3.5-turbo-1106",
  response_format={"type": "json_object"},
  messages=[
    {"role": "system", "content": "You are a helpful programmer who always returns your answer in JSON."},
    {"role": "user", "content": "give me a list of 5 things for grocery shopping. call the list 'groceries'"}
  ]
)

print(completion.choices[0].message)
print(completion)


ChatCompletionMessage(content='{\n  "groceries": [\n    "eggs",\n    "milk",\n    "bread",\n    "apples",\n    "chicken"\n  ]\n}', role='assistant', function_call=None, tool_calls=None)
ChatCompletion(id='chatcmpl-8L9G16X0T1VcGC6AMYIEPtPsXApw3', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='{\n  "groceries": [\n    "eggs",\n    "milk",\n    "bread",\n    "apples",\n    "chicken"\n  ]\n}', role='assistant', function_call=None, tool_calls=None))], created=1700051165, model='gpt-3.5-turbo-1106', object='chat.completion', system_fingerprint='fp_eeff13170a', usage=CompletionUsage(completion_tokens=34, prompt_tokens=43, total_tokens=77))


In [10]:
import json
# Use json.dumps() to pretty print the JSON
pretty_json = json.dumps(completion.choices[0].message, indent=4)

# Print the pretty JSON
print(pretty_json)

TypeError: Object of type ChatCompletionMessage is not JSON serializable

DALL-E 3

In [3]:
PROMPT = "a room full of dogs all meditating in a circle"

In [4]:
from openai import OpenAI
client = OpenAI()

response = client.images.generate(
  model="dall-e-3",
  prompt= PROMPT,
  size="1024x1024",
  quality="standard",
  n=1,
)

image_url = response.data[0].url
image_url

'https://oaidalleapiprodscus.blob.core.windows.net/private/org-3Ei0iqcWZOIg7fs9QRxa4srN/user-TAKHWC1qaO1ZBXgUaVY8eM6I/img-JvlJhR9umt7sUga8SUJpmNYf.png?st=2023-11-15T11%3A28%3A45Z&se=2023-11-15T13%3A28%3A45Z&sp=r&sv=2021-08-06&sr=b&rscd=inline&rsct=image/png&skoid=6aaadede-4fb3-4698-a8f6-684d7786b067&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2023-11-14T23%3A29%3A26Z&ske=2023-11-15T23%3A29%3A26Z&sks=b&skv=2021-08-06&sig=X%2BsZid/R5ZhTgj/u1Qnw5vMnvcXlbgltQCrlUI3jyjM%3D'

In [5]:
import requests
from PIL import Image
from io import BytesIO
from IPython.display import display

def display_and_save_image_from_url(url, scale_percent=100, save_name='image.png'):
    # Send a GET request to the specified URL to retrieve the image
    response = requests.get(url)
    # Open the image
    img = Image.open(BytesIO(response.content))

    # Calculate the new size, as a percentage of the original size
    if scale_percent != 100:
        width, height = img.size
        new_width = int(width * scale_percent / 100)
        new_height = int(height * scale_percent / 100)
        img = img.resize((new_width, new_height), Image.ANTIALIAS)

    # Save the image locally with the given name
    img.save(save_name)

    # Display the image in the notebook
    display(img)

# Call the function with the URL, the scale percentage, and the save name you want
display_and_save_image_from_url(image_url, scale_percent=50, save_name='../data/meditating_dogs.png')

AttributeError: module 'PIL.Image' has no attribute 'ANTIALIAS'

GPT Voice


In [ ]:
import base64
import requests

# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# Path to your image
image_path = "../data/meditating_dogs.png"

# Getting the base64 string
base64_image = encode_image(image_path)

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {os.environ['OPENAI_API_KEY']}"
}

payload = {
    "model": "gpt-4-vision-preview",
    "messages": [
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": "Describe what’s in this image in detail as a story?"
          },
          {
            "type": "image_url",
            "image_url": {
              "url": f"data:image/jpeg;base64,{base64_image}"
            }
          }
        ]
      }
    ],
    "max_tokens": 300
}

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

print(response.json())

In [ ]:
story = response.json()['choices'][0]['message']['content']

print(story)

## Text To Speech


In [ ]:
from IPython.display import Audio
from pathlib import Path

response = client.audio.speech.create(
  model="tts-1",
  voice="onyx",
  input=story
)

# Define the path where you want to save the file
speech_file_path = Path('../data/story.mp3')

# Save the response content (binary content of the mp3 file) to the path
with open(speech_file_path, 'wb') as file:
    file.write(response.content)

# Play the audio file
Audio(speech_file_path, autoplay=True)
